# Wide-field imaging demonstration

### This script makes a fake data set, fills it with a number of point components, and then images it. 

In [ ]:
%matplotlib inline

# Wide-field imaging demonstration

### This script makes a fake data set, fills it with a number of point components, and then images it. 

# In[1]:

import os
import sys

sys.path.append(os.path.join('..', '..'))

from matplotlib import pylab

pylab.rcParams['figure.figsize'] = (12.0, 12.0)
pylab.rcParams['image.cmap'] = 'rainbow'

import numpy

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy import constants as const
from astropy.wcs.utils import pixel_to_skycoord

from matplotlib import pyplot as plt

from arl.visibility.operations import create_visibility
from arl.skymodel.operations import create_skycomponent
from arl.image.operations import show_image, export_image_to_fits
from arl.util.testing_support import create_named_configuration
from arl.fourier_transforms.ftprocessor import invert_2d, create_image_from_visibility, \
    weight_visibility, predict_skycomponent_visibility, create_w_term_image, invert_image_partition

import logging

log = logging.getLogger()
log.setLevel(logging.INFO)
log.addHandler(logging.StreamHandler(sys.stdout))

doplot = True

### Construct the SKA1-LOW core configuration

In [ ]:
params = {'npixel': 512,
          'cellsize': 0.001,
          'spectral_mode': 'channel',
          'channelwidth': 5e7,
          'reffrequency': 1e8,
          'image_partitions': 5}

vlaa = create_named_configuration('LOWBD2-CORE')

### We create the visibility. This just makes the uvw, time, antenna1, antenna2, weight columns in a table

In [ ]:
times = numpy.arange(-numpy.pi / 2.0, +numpy.pi / 2.0, 0.05)
frequency = numpy.array([1e8])

reffrequency = numpy.max(frequency)
phasecentre = SkyCoord(ra=+15.0 * u.deg, dec=-35.0 * u.deg, frame='icrs', equinox=2000.0)
vt = create_visibility(vlaa, times, frequency, weight=1.0, phasecentre=phasecentre)
uvw = vt.data['uvw']

### Plot the synthesized UV coverage.

In [ ]:
if doplot:
    plt.clf()
    for f in frequency:
        x = f / const.c
        plt.plot(x * vt.data['uvw'][:, 0], x * vt.data['uvw'][:, 1], '.', color='b')
        plt.plot(-x * vt.data['uvw'][:, 0], -x * vt.data['uvw'][:, 1], '.', color='r')
        plt.xlabel('U (wavelengths)')
        plt.ylabel('V (wavelengths)')

### To see why this occurs, look at the phase term due to w. Evaluate this for the median absolute w. 

In [ ]:
if doplot:
    wterm = create_w_term_image(vt, params=params)
    show_image(wterm)
    plt.show()

### Create a grid of components and predict each in turn, using the full phase term including w.

In [ ]:
flux = numpy.array([[100.0, 0.0, 0.0, 0.0]])
vt.data['vis'] *= 0.0

model = create_image_from_visibility(vt, params=params)

spacing_pixels = params['npixel'] // params['image_partitions']
log.info('Spacing in pixels = %s' % spacing_pixels)
spacing = 180.0 * params['cellsize'] * spacing_pixels / numpy.pi
centers = -2.0, -1.0, 0.0, 1.0, 2.0
for iy in centers:
    for ix in centers:
        pra, pdec = params['npixel'] // 2 + ix * spacing_pixels,\
                    params['npixel'] // 2 + iy * spacing_pixels
        sc = pixel_to_skycoord(pra, pdec, model.wcs)
        log.info("Component at (%f, %f) %s" % (pra, pdec, str(sc)))
        comp = create_skycomponent(flux=flux, frequency=frequency, direction=sc)
        predict_skycomponent_visibility(vt, comp)

### Make the dirty image and point spread function. Note that the shape of the sources vary with position in the image. This space-variant property of the PSF arises from the w-term.

In [ ]:
dirty = create_image_from_visibility(vt, params=params)
psf = create_image_from_visibility(vt, params=params)
vt = weight_visibility(vt, dirty)
dirty = invert_2d(vt, dirty, params=params)
psf = invert_2d(vt, dirty, dopsf=True, params=params)
psfmax = psf.data.max()
dirty.data = dirty.data / psfmax
psf.data = psf.data / psfmax

if doplot:
    show_image(dirty)

print("Max, min in dirty image = %.6f, %.6f, psfmax = %f" % (dirty.data.max(), dirty.data.min(), psfmax))
print("Max, min in PSF = %.6f, %.6f, psfmax = %f" % (psf.data.max(), psf.data.min(), psfmax))

export_image_to_fits(dirty, 'imaging-wterm_dirty.fits')
export_image_to_fits(psf, 'imaging-wterm_psf.fits')

### Use image plane partitioning (faceting) to make the image

In [ ]:
dirtyFacet = create_image_from_visibility(vt, params=params)
dirtyFacet = invert_image_partition(vt, dirtyFacet, params=params)
dirtyFacet.data = dirtyFacet.data / (psfmax * params['image_partitions'] * params['image_partitions'])

if doplot:
    show_image(dirtyFacet)

print("Max, min in dirty image = %.6f, %.6f, psfmax = %f" % (dirtyFacet.data.max(), dirtyFacet.data.min(), psfmax))
export_image_to_fits(dirtyFacet, 'imaging-wterm_dirtyFacet.fits')

### That was the best case. This time, we will not arrange for the points to be at the center of the partitions.

In [ ]:
params['image_partitions']=7
dirtyFacet = create_image_from_visibility(vt, params=params)
dirtyFacet = invert_image_partition(vt, dirtyFacet, params=params)
dirtyFacet.data = dirtyFacet.data / (psfmax * params['image_partitions'] * params['image_partitions'])

if doplot:
    show_image(dirtyFacet)

print("Max, min in dirty image = %.6f, %.6f, psfmax = %f" % (dirtyFacet.data.max(), dirtyFacet.data.min(), psfmax))
export_image_to_fits(dirtyFacet, 'imaging-wterm_dirtyFacet_7.fits')

### As expected for this high Fresnel number example, a moderate number of facets leaves very substantial errors in the image.
